<a href="https://colab.research.google.com/github/abbasKareem/SNR/blob/master/snr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
from scipy.stats import norm
import scipy.io
from scipy import linalg

np.random.seed(42)



In [2]:
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

In [ ]:
Ns=1000

SNR =np.array([-20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0], dtype='int64')
snr = np.array([0.0100, 0.0126, 0.0158, 0.0200, 0.0251, 0.0316, 0.0398,  0.0501, 0.0631, 0.0794, 0.1000, 0.1259, 0.1585, 0.1995,  0.2512, 0.3162, 0.3981, 0.5012 ,0.6310, 0.7943, 1.0000], dtype='float64')
L=3
nn=0
m=0
pf=0.01
num_iter=10
l=1
test = 0
# for x in (n+1 for n in range(5)):
for i in SNR:
  count = 0
  conf = 0
  for h in (n+1 for n in range(num_iter)):
    d = np.array([1, 0, 1, 1, 0]) # Data sequence
    b = (2 * d) - 1 # Convert unipolar to bipolar
    T = 1 # Bit duration
    Eb = T/2 # This will result in unit amplitude waveforms
    fc = 7/T # Carrier frequency
    t = np.linspace(0, 5, Ns) # discrete time sequence between 0 and 5*T (1000 samples)
    N = len(t) # Number of samples
    Nsb = N/len(d) # Number of samples per bit
    x = d.conj().T
    y = (1, Nsb)
    dd = np.tile(x, (1, int(Nsb))) # replicate each bit Nsb times
    bb = np.tile(b.conj().T,(1, int(Nsb)))  # Transpose the rows and columns
    dw = dd.conj().transpose()
    dw = np.concatenate([dw.flat]) # Convert dw to a column vector (colum by column) and convert to a row vector
    bw = bb.conj().T # line 32
    bw = np.concatenate([bw.flat]) # Data sequence samples line 33  
    w = np.dot(np.sqrt((np.dot(2,Eb))/T), np.cos(np.dot(np.dot(np.dot(2, math.pi), fc), t))) # carrier waveform line 34
    bpsk_w = bw*w # modulated waveform line 35
    signal = bpsk_w # line 36
    noise = np.random.standard_normal((Ns,)) # line 37
    noise_power= (np.linalg.norm(noise)) ** 2 # line 38
    signal_power = (np.linalg.norm(signal)) ** 2 # line 39
    # mult = sqrt(snr(i)*noise_power/signal_power) # line 40
    mult =np.sqrt(((signaltonoise(np.array([i]))) * noise_power) / signal_power) # line 40
    signal1 =np.dot(mult, signal) # line 41
    signal2 = signal1 + noise # line 42
    k = 0 # line 43
    Cx = np.zeros((1, L)) # line 44
    for n in range(L - 1): # line 45
      for j in range(1, (Ns-L-1), 1): # line 46
        Cx[:, n] = Cx[:, n] + (signal2[j] * signal2[j+k]) # line 47
        print(j)
      k = k +1 # line 49
    nvar = noise.var(axis=0) # line 51
    svar = signal.var(axis=0) # line 52
    sn = svar / nvar #line 53
    Cx = Cx / Ns # line 54
    Cx_mtx = linalg.toeplitz(Cx) # line 55
    T2 = np.sum(np.diag(np.abs(Cx_mtx))) # line 56
    T1 = np.sum(np.sum(np.abs(Cx_mtx))) # line 57
    y=T1 / T2 #line 58
    nu = 1.585 #line 59
    Rl = (T1 - T2) / (sn*nvar*nu) # line 60
    a0 = (Rl*sn) / (sn+1) # line 61
    a1 = np.sqrt(2 / (Ns * math.pi)) # line 62
    a2 = L - 1  # line 63
    a = (a2 * a1) - a0 # line 64
    b = ((a2 * a1) - a0) + 2 # line 65
    c0 = (1 - pf) / pf # line 66
    c1=4/Ns # line 67
    c2 = 1 + (a2*a1) # line 68
    c3 = a0 + 1 # line 69
    Aa = np.log(c0 * (c2 / c3)) # line 70
    A = Aa * c1 #line 71
    t1 = 1 + ((A*b)/a) # line 72
    th1 = (a/A)*(-1 +np.sqrt(t1)) # line 73
    R2 = (T1 - T2) /((sn/(nvar*nu))) # line 74
    aa0 = (R2 * sn) / (sn + 1) # line 75
    aa = (a2 * a1) -aa0 # line 76
    bb = (a2 * a1) -aa0 + 2 # line 77
    cc3 = aa0 + 1 # line 78
    Aaa =np.log(c0*(c2/cc3)) # line 79
    AA = Aaa * c1  # line 80
    t2 = 1 + ((AA * bb) / aa) # line 81
    th2 = (aa / AA) * (-1 + np.sqrt(t2)) # line 82
    




# y = x.flatten(1)

In [5]:
c2 = 1.0505
c3 = 3.3016
c2 / c3

0.318179064695905